<a href="https://colab.research.google.com/github/a-matson/DST_ASS1/blob/main/report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To-do



* Research, discuss and select performance metrics - ALL
* Introduce the dataset - ?
* EDA - Josh
* Everyone do their model experiments
  * Logistic regression - Geraint
  * SVM - Andre
  * Neural networks - Ethan
  * Decision trees - Josh
* Summary/conclusion - ?



# Introduction

Cite introductory paper for dataset: https://www.semanticscholar.org/paper/Nuclear-feature-extraction-for-breast-tumor-Street-Wolberg/53f0fbb425bc14468eb3bf96b2e1d41ba8087f36

All samples are taken from breast tumours. For each sample (fine needle aspirate), they use computer vision techniques to identify all of the cell nuclei and compute 10 features for each nucleus. The dataset contains, for each sample, the mean, largest (or 'worst') and standard error of each feature.

https://ftp.cs.wisc.edu/math-prog/cpo-dataset/machine-learn/cancer/WDBC/WDBC.dat

# Performance metrics

As researched (and discussed in dataset paper), medical performance metrics are sensitivity and specificity.

Our primary goal will be maximising sensitivity values but we will also report specificity/ ROC curves/ accuracy etc.

This is because we want to penalise false negatives as much as possible, as missing a cancer diagnosis is potentially fatal.

# Imports and setting up dataset

In [ ]:
!pip install ucimlrepo

In [ ]:
import sklearn as sklearn

from sklearn.model_selection import train_test_split

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

# data (as pandas dataframes)
X = breast_cancer_wisconsin_diagnostic.data.features
y = breast_cancer_wisconsin_diagnostic.data.targets

y['Diagnosis'] = y['Diagnosis'].map({'M':0, 'B':1})

# metadata
#print(breast_cancer_wisconsin_diagnostic.metadata)

# variable information
#print(breast_cancer_wisconsin_diagnostic.variables)

<ipython-input-13-411e26e520e5>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['Diagnosis'] = y['Diagnosis'].map({'M':0, 'B':1})


In [ ]:
column_name_mapping = {
    "radius1": "radius_mean",
    "texture1": "texture_mean",
    "perimeter1": "perimeter_mean",
    "area1": "area_mean",
    "smoothness1": "smoothness_mean",
    "compactness1": "compactness_mean",
    "concavity1": "concavity_mean",
    "concave_points1": "concave points_mean",
    "symmetry1": "symmetry_mean",
    "fractal_dimension1": "fractal_dimension_mean",
    "radius2": "radius_se",
    "texture2": "texture_se",
    "perimeter2": "perimeter_se",
    "area2": "area_se",
    "smoothness2": "smoothness_se",
    "compactness2": "compactness_se",
    "concavity2": "concavity_se",
    "concave_points2": "concave points_se",
    "symmetry2": "symmetry_se",
    "fractal_dimension2": "fractal_dimension_se",
    "radius3": "radius_worst",
    "texture3": "texture_worst",
    "perimeter3": "perimeter_worst",
    "area3": "area_worst",
    "smoothness3": "smoothness_worst",
    "compactness3": "compactness_worst",
    "concavity3": "concavity_worst",
    "concave_points3": "concave points_worst",
    "symmetry3": "symmetry_worst",
    "fractal_dimension3": "fractal_dimension_worst",
}


In [ ]:
# Rename columns

X = X.rename(columns=column_name_mapping)
X.columns

Index(['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')

In [ ]:
# Train-test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

In [ ]:
X_train.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
342,11.06,14.96,71.49,373.9,0.10330,0.09097,0.05397,0.03341,0.1776,0.06907,...,11.92,19.90,79.76,440.0,0.1418,0.2210,0.2299,0.10750,0.3301,0.09080
138,14.95,17.57,96.85,678.1,0.11670,0.13050,0.15390,0.08624,0.1957,0.06216,...,18.55,21.43,121.40,971.4,0.1411,0.2164,0.3355,0.16670,0.3414,0.07147
434,14.86,16.94,94.89,673.7,0.08924,0.07074,0.03346,0.02877,0.1573,0.05703,...,16.31,20.54,102.30,777.5,0.1218,0.1550,0.1220,0.07971,0.2525,0.06827
47,13.17,18.66,85.98,534.6,0.11580,0.12310,0.12260,0.07340,0.2128,0.06777,...,15.67,27.95,102.80,759.4,0.1786,0.4166,0.5006,0.20880,0.3900,0.11790
64,12.68,23.84,82.69,499.0,0.11220,0.12620,0.11280,0.06873,0.1905,0.06590,...,17.09,33.47,111.80,888.3,0.1851,0.4061,0.4024,0.17160,0.3383,0.10310


In [ ]:
y_train.head()

,Diagnosis
342,1
138,0
434,1
47,0
64,0


# Andre

Structure:
- state high correlation (which is expected with variable relation)
- consider ration of diagnoses M vs B
- Look at y's spread (boxplot or something more sophisticated like violin plot which may be the most appropriate)
- This can lead to conclusion based on bivariate distributions with diagnosis that larger values correlate with getting diagnose as M.
- This in turn allows us to evaluate variables that aren't strongly correlated with the outcome. (which are: `'fractal_dimension_mean', 'texture_se', 'smoothness_se', 'symmetry_se', 'fractal_dimension_se'`)
- We can test feature relationship with the target variable by using one-way ANOVA (but this may be excessive since visualisation already show significant idications)
- There is a lot of difference in terms of the scales of the predictors. (ie the area and perimeter) This requires us to normalise data.
- Analyse data... (31 predictors)
- This data highly dimensional and highly correlated within predictors
- Logistic regression may be used (since there's not much data but a lot of features)
- Maybe justify in more details what corrections could be used to improve its predictions
- SVC is less sensitive to multicollinearity.


"VIF measures the extent to which each predictor is estimateable by the other predictors, measuring the extent of linear dependence of each predictor by the others. We use the function from statsmodels"

### Scale data

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

testing scaling on logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
lr = LogisticRegression()
lr.fit(X_train_scaled, y_train)
pred_lr = lr.predict(X_test_scaled)
print(classification_report(y_test, pred_lr, target_names=['M', 'B']))

              precision    recall  f1-score   support

           M       0.89      1.00      0.94        34
           B       1.00      0.95      0.97        80

    accuracy                           0.96       114
   macro avg       0.95      0.97      0.96       114
weighted avg       0.97      0.96      0.97       114

1.0


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## SMV

explain why we do SVC

In [ ]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train_scaled, y_train)
pred_svc = svc.predict(X_test_scaled)
print(classification_report(y_test, pred_svc, target_names=['M', 'B']))

              precision    recall  f1-score   support

           M       0.92      1.00      0.96        34
           B       1.00      0.96      0.98        80

    accuracy                           0.97       114
   macro avg       0.96      0.98      0.97       114
weighted avg       0.98      0.97      0.97       114



/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


oh cool basic svc seems to be better than most default lr

just checking if it's not because default rbf kernal used by default svc so will try linear

In [ ]:
svc_new = SVC(kernel='linear')
svc_new.fit(X_train_scaled, y_train)
pred_svc_new = svc_new.predict(X_test)
print(classification_report(y_test, pred_svc_new, target_names=['M', 'B']))

              precision    recall  f1-score   support

           M       0.30      1.00      0.46        34
           B       0.00      0.00      0.00        80

    accuracy                           0.30       114
   macro avg       0.15      0.50      0.23       114
weighted avg       0.09      0.30      0.14       114



/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave

oh wow this is on the opposite very very bad but sorta expected

# Ethan

#Josh

#Geraint